# Singleton Allocator

> Implementing a singleton by rewriting the class allocator.

An **allocator** is a tool or mechanism used to allocate (reserve), manage, and release memory dynamically during the lifetime of a program.

In Python, the `__new__` method is responsible for creating and returning a new instance of the class it belongs to. One of its responsibilities is **allocating memory** for instantiating the class.

> Do not confure `__new__` with `__init__`: `__new__` is called first and creates the object, and then `__init__` is called to initialize it.

We can override the `__new__` method of a class in order to convert it into a singleton. For this example, let's pretend we're building a database that we only want to load once: 

In [1]:
class Database:
    _instance = None # static instance
    
    def __new__(cls, *args, **kwargs):
        """We check whether or not some static instance has already been created, and we either return the newly created instance or the already existing one"""
        if not cls._instance:
            # We create a new instance by calling the original __new__ method
            cls._instance = super(Database, cls).__new__(cls, *args, **kwargs)
        return cls._instance

So, we simply control whether an instance already exists or not, and we either create it and return it, or return the previously existing instance.

Let's see it in action:

In [2]:
d1 = Database()
d2 = Database()
print(d1 == d2)

True


This seems to work, but we may actually encounter some problems when we try to initialize the object. Let's try initializing our database:

In [5]:
import random

class Database:
    _instance = None # static instance
    
    def __new__(cls, *args, **kwargs):
        """We check whether or not some static instance has already been created, and we either return the newly created instance or the already existing one"""
        if not cls._instance:
            # We create a new instance by calling the original __new__ method
            cls._instance = super(Database, cls).__new__(cls, *args, **kwargs)
        return cls._instance
    
    def __init__(self):
        self.id = random.randint(1,101)
        print(f'Loading a database from file, code {self.id}')

In [6]:
d1 = Database()
d2 = Database()
print(d1 == d2)

Loading a database from file, code 60
Loading a database from file, code 32
True


`__init__` is always called right after `__new__`. Even if we're not creating a second instance and both `d1` and `d2` are referencing the same instance, when we call `Database()` we also call `__new__` and `__init__` is always called automatically afterwards; as a result, our singleton instance is being initialized twice.

We need to add some sort of guard in order to prevent this, which we will see in the next lesson.